In [3]:
import torch
a = torch.load("/afs/crc.nd.edu/group/dmsquare/vol2/myu2/ComparisonSentences/experiments/multinews_extractive/bert1213/checkpoint-50000/pred/predictions.pt")

/afs/crc.nd.edu/user/m/myu2/anaconda2/envs/bert/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
a = torch.Tensor(a)
a.shape

torch.Size([5622, 100, 2])

In [5]:
a[0]

tensor([[-2.6590,  2.2262],
        [ 0.8407, -0.7048],
        [-1.9699,  1.8817],
        [-3.9118,  3.7355],
        [-0.5102,  0.1573],
        [-1.0450,  0.8167],
        [-0.7691,  0.4873],
        [-1.0002,  0.5313],
        [ 4.9005, -4.2765],
        [ 4.6939, -4.0267],
        [ 5.0705, -4.5717],
        [ 4.7987, -4.2977],
        [ 5.2911, -4.8533],
        [ 5.0977, -4.5027],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1

In [6]:
from scipy.special import softmax
b = softmax(a, axis=-1)
c = b[:,:,1]

In [7]:
c[0]

tensor([9.9250e-01, 1.7573e-01, 9.7920e-01, 9.9952e-01, 6.6095e-01, 8.6550e-01,
        7.7841e-01, 8.2224e-01, 1.0338e-04, 1.6317e-04, 6.4928e-05, 1.1206e-04,
        3.9294e-05, 6.7696e-05, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01,
        5.0000e-01, 5.0000e-01, 5.0000e-

In [8]:
from numpy import argsort
argsort(c)

tensor([[12, 10, 13,  ...,  2,  0,  3],
        [ 3,  0,  5,  ..., 11, 10, 12],
        [ 4,  5,  7,  ...,  9,  1,  2],
        ...,
        [11,  5,  4,  ...,  6,  1,  8],
        [ 8, 14,  4,  ..., 44,  2,  0],
        [ 1, 17, 13,  ..., 11,  8,  5]])

In [22]:
import json
import numpy as np
from numpy import argsort
predictions = [] 
references = []
with open("/afs/crc.nd.edu/group/dmsquare/vol2/myu2/ComparisonSentences/downloaded/multi_doc_summ/matchsum_multi-news/test_multinews_brief.json") as f:
    topk = 6
    for idx,line in enumerate(f):
        if idx == 10:
            break
        obj = json.loads(line)
        labels = obj["label"]
        sent_ids = np.arange(0, len(labels), 1)
        scores = c[idx][sent_ids]
        print(scores)
        rank_ids = argsort(scores)[:topk] if len(labels) > topk else argsort(scores)
        # print(rank_ids)
        selected_sentences = [obj["text"][x] for x in rank_ids]
        predictions.append(' '.join(selected_sentences))
        references.append(' '.join(obj['summary']))

print(len(predictions), len(references))
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge1_all = []
rouge2_all = []
rougeL_all = []
cnt_empty = 0
for idx, gold in enumerate(references):

    scores = scorer.score(predictions[idx] , gold)
    rouge1_all.append(scores["rouge1"][2])
    rouge2_all.append(scores["rouge2"][2])
    rougeL_all.append(scores["rougeL"][2])
print(sum(rouge1_all)/len(rouge1_all), sum(rouge2_all)/len(rouge2_all), sum(rougeL_all)/len(rougeL_all),)


        
        
        


tensor([9.9250e-01, 1.7573e-01, 9.7920e-01, 9.9952e-01, 6.6095e-01, 8.6550e-01,
        7.7841e-01, 8.2224e-01, 1.0338e-04, 1.6317e-04, 6.4928e-05, 1.1206e-04,
        3.9294e-05, 6.7696e-05, 5.0000e-01])
tensor([5.2710e-04, 3.1383e-03, 2.1789e-01, 2.9400e-04, 1.1851e-02, 2.4038e-03,
        1.1641e-01, 1.1066e-01, 1.8770e-01, 1.8584e-01, 9.3332e-01, 8.8772e-01,
        9.9658e-01, 5.0000e-01, 5.0000e-01])
tensor([1.1363e-03, 9.2424e-01, 9.8880e-01, 7.8834e-03, 7.3673e-05, 1.1117e-04,
        1.2644e-02, 7.6979e-04, 2.8396e-03, 9.1921e-01, 3.5944e-01, 8.9281e-01,
        7.2866e-01, 8.5345e-01, 8.3935e-02, 5.0000e-01])
tensor([4.2838e-03, 8.3443e-01, 7.1176e-01, 3.5169e-01, 2.5701e-01, 9.4964e-03,
        6.0890e-01, 7.1853e-04, 1.8999e-01, 2.5678e-01, 6.2233e-02, 4.8553e-04,
        4.7325e-01, 9.9929e-01, 9.6767e-02, 1.7511e-02, 5.0000e-01, 5.0000e-01,
        5.0000e-01])
tensor([4.5288e-02, 9.9388e-01, 8.7500e-02, 1.9862e-02, 1.6219e-01, 8.1040e-01,
        3.2069e-04, 1.9747e-01, 

In [23]:
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


In [31]:
text = "los angeles ( ap ) \u2014 in her first interview since the nba banned her estranged husband , shelly sterling says she will fight to keep her share of the los angeles clippers and plans one day to divorce donald sterling ."
print(len(tokenizer.encode(text)))
print(len(text.split()))

56
42
